In [1]:
!pip install -q -U transformers datasets peft trl bitsandbytes accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.5/465.5 kB 40.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.9/380.9 kB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 18.0 MB/s eta 0:00:00


In [29]:
base_model = "TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T"

In [30]:
tokenizer = AutoTokenizer.from_pretrained(base_model)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

In [31]:
import zipfile
import os
# Path to your zip file
zip_path = "/content/tinyllama-instruction.zip"

# Extract all files
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall()

In [32]:
model_path = "/content/checkpoint-3"

In [33]:
instruction_model = AutoModelForCausalLM.from_pretrained(model_path, device_map="auto")

In [37]:
prompt = "Explain how artificial intelligence is improving the process of drug discovery and development in the pharmaceutical industry."
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

outputs = instruction_model.generate(
    **inputs,
    max_new_tokens=100,
    temperature=0.8,
    top_p=0.9,
    do_sample=True,
    repetition_penalty=1.1
)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Explain how artificial intelligence is improving the process of drug discovery and development in the pharmaceutical industry.
Teaching Strategies for AI in Pharmacy: Developing and delivering teaching activities that encourage the use of artificial intelligence (AI) to enhance students' knowledge, skills and understanding will be discussed.
Artificial Intelligence: AI has been used in many sectors of our society such as healthcare, manufacturing, education and the military. These applications have created an awareness and interest amongst students that they can apply AI within their care


In [38]:
from trl import DPOTrainer
from transformers import AutoTokenizer,  AutoModelForCausalLM, TrainingArguments
from peft import PeftModel
from peft import LoraConfig, get_peft_model, TaskType
from datasets import load_dataset
import torch

In [39]:
base_model = "TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T"

In [40]:
instruction_checkpoint = "/content/checkpoint-3"

In [41]:
dataset = load_dataset("csv", data_files="/content/pharma_preference_data.csv")["train"]

In [42]:
tokenizer = AutoTokenizer.from_pretrained(base_model)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

In [43]:
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    target_modules=["q_proj", "v_proj"],
    bias="none"
)

In [44]:
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    load_in_8bit=True,
    device_map="auto"
)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


In [45]:
model = PeftModel.from_pretrained(model, instruction_checkpoint)

In [46]:
model = model.merge_and_unload()

/usr/local/lib/python3.12/dist-packages/peft/tuners/lora/bnb.py:110: UserWarning: Merge lora module to 8-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [47]:
pref_model_lora = get_peft_model(model, lora_config)

/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:282: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


In [48]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [49]:
from trl import DPOTrainer, DPOConfig

In [50]:
dpo_args = DPOConfig(
    output_dir="./tinyllama-preference-alignment",
    learning_rate=2e-5,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    num_train_epochs=1,
    beta=0.1,
    report_to=None,
    logging_dir=None,
    loss_type="sigmoid",
    remove_unused_columns=False
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [51]:
trainer = DPOTrainer(
    model=pref_model_lora,
    ref_model=None,
    args=dpo_args,
    train_dataset=dataset,
    processing_class=tokenizer,
)

In [52]:
trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 2}.
/usr/local/lib/python3.12/dist-packages/bitsandbytes/autograd/_functions.py:181: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.12/dist-packages/bitsandbytes/autograd/_functions.py:181: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.12/dist-packages/torch/utils/checkpoint.py:85: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Step,Training Loss


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


TrainOutput(global_step=1, training_loss=0.66193026304245, metrics={'train_runtime': 4.1452, 'train_samples_per_second': 1.206, 'train_steps_per_second': 0.241, 'total_flos': 0.0, 'train_loss': 0.66193026304245, 'epoch': 1.0})

In [54]:
model_path = "/content/checkpoint-3"
instruction_model = AutoModelForCausalLM.from_pretrained(model_path, device_map="auto")

inputs = tokenizer(question, return_tensors="pt").to("cuda")

outputs = instruction_model.generate(
    **inputs,
    max_new_tokens=100,
    temperature=0.8,
    top_p=0.9,
    do_sample=True,
    repetition_penalty=1.1
)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Explain how Metformin works in the human body and why some researchers believe it could have benefits beyond diabetes treatment.
Crohn's disease (CD) is an inflammatory bowel disease that can be severe. Recent research has shown that metformin may reduce the risk of flare-ups in people with CD.
What Is Crohn's Disease?
Crohn's disease (CD) is a type of inflammatory bowel disease that is characterized by inflammation in the intestinal tract, which can lead to abdom


In [56]:
model_path = "/content/tinyllama-preference-alignment/checkpoint-1"
preference_aligned_model = AutoModelForCausalLM.from_pretrained(model_path, dtype=torch.float16)

In [57]:
preference_aligned_model.to("cuda")

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 2048)
    (layers): ModuleList(
      (0-21): 22 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): lora.Linear(
            (base_layer): Linear(in_features=2048, out_features=2048, bias=False)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.05, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=2048, out_features=8, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=8, out_features=2048, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
            (lora_magnitude_vector): ModuleDict()
          )
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): lora.Linear(
            (base_layer): Linear(in_features=2048, out_features=256, bia

In [58]:
inputs = tokenizer(question, return_tensors="pt").to("cuda")
outputs = preference_aligned_model.generate(
    **inputs,
    max_new_tokens=100,
    temperature=0.8,
    top_p=0.9,
    do_sample=True,
    repetition_penalty=1.1
)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Explain how Metformin works in the human body and why some researchers believe it could have benefits beyond diabetes treatment.
Metformin, a common diabetes drug used to treat type 2 diabetes, has long been known as a "fat burner," but new research suggests that it may also help prevent cancer.
In a review of more than 100 studies published this week in the journal Diabetologia, researchers from Britain's National Health Service (NHS) and Oxford University said metformin may have properties that can be exploited to prevent cancer by:
